In [1]:
import os
print(os.getcwd())

e:\YangYueHan\project\MCLP\model


In [2]:
dataset_path = '../data/MP'

In [3]:
from tools import get_mapper

In [5]:
get_mapper(dataset_path=dataset_path)


*** Please check the corresponding dataset configuration yml file. ***
unique location num: 20607
unique user num: 10000


In [6]:
import numpy as np


- np.load()
- data_structure: item()

In [7]:
location_mapper = np.load(os.path.join(dataset_path, 'location_mapper.npy'), allow_pickle=True).item()

- dict: items()

In [8]:
for i, (loc, idx) in enumerate(list(location_mapper.items())[:5]):
    print(f'{i}: {loc} -> {idx}')

0: 9605 -> 0
1: 2085 -> 1
2: 6539 -> 2
3: 12587 -> 3
4: 12500 -> 4


In [1]:
from dataloader import datetime_to_features

In [2]:
weekday, hour = datetime_to_features(1672567200000)

In [3]:
print(weekday)
print(hour)

6
18
